In [133]:
import keras
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Dropout
from keras.layers import Dense

import numpy as np
import pandas as pd
np.random.seed(1)

In [25]:
#Importing raw data
rawData=pd.read_csv('./Raw Data/ign.csv',index_col=0)

In [26]:
#Grouping 'score_phrase' to represent just two emotions (positive or negative)
def binaryEmotions(emotion):
    if (emotion in ['Amazing', 'Great', 'Good', 'Masterpiece', 'Okay']):
        return 1
    else:
        return 0

rawData['binEmotions']=rawData.apply(lambda row: binaryEmotions(row['score_phrase']),axis=1)

In [82]:
#Separating features and labels
features=rawData['title']
labels=pd.Categorical(rawData['score_phrase'])
labelsBin=pd.Categorical(rawData['binEmotions'])

In [83]:
#Converting labels to vectors and one-hot encoding feature sentences
labelsVector=np_utils.to_categorical(labels.codes)
labelsBinVector=np_utils.to_categorical(labelsBin.codes)

featuresList=features.apply(lambda x: one_hot(x,10000))

In [92]:
#Getting maximum sentence length and padding with maxlen of 30
print("Maximum sentence length in raw data =",max([len(x) for x in featuresList]))
featuresPad=pad_sequences(featuresList,maxlen=30,value=0.)

Maximum sentence length in raw data = 17


In [135]:
#Create model - for 11 emotions model
model=Sequential()
model.add(Embedding(input_dim=10000,input_length=30,output_dim=256))
model.add(LSTM(256,dropout_U=0.5,dropout_W=0.5))
model.add(Dense(11,activation='softmax'))

In [137]:
#Compile model
adam=Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

#Train the model
model.fit(featuresPad,labelsVector,batch_size=10,nb_epoch=50,validation_split=0.1)

Train on 16762 samples, validate on 1863 samples
Epoch 1/50
16762/16762 [==============================] - 388s - loss: 1.8903 - acc: 0.2533 - val_loss: 1.8169 - val_acc: 0.2979
Epoch 2/50
16762/16762 [==============================] - 387s - loss: 1.8695 - acc: 0.2694 - val_loss: 1.8228 - val_acc: 0.2555
Epoch 3/50
16762/16762 [==============================] - 387s - loss: 1.8329 - acc: 0.3078 - val_loss: 1.8024 - val_acc: 0.2856
Epoch 4/50
16762/16762 [==============================] - 390s - loss: 1.7585 - acc: 0.3486 - val_loss: 1.8146 - val_acc: 0.2818
Epoch 5/50
16762/16762 [==============================] - 391s - loss: 1.6581 - acc: 0.4051 - val_loss: 1.8728 - val_acc: 0.2797
Epoch 6/50
16762/16762 [==============================] - 394s - loss: 1.5653 - acc: 0.4446 - val_loss: 1.9187 - val_acc: 0.2850
Epoch 7/50
  610/16762 [>.............................] - ETA: 381s - loss: 1.4629 - acc: 0.4820

KeyboardInterrupt: 

In [ ]:
#Binary emotion model
modelBin=Sequential()
modelBin.add(Embedding(input_dim=10000,input_length=30,output_dim=256))
modelBin.add(LSTM(256))
modelBin.add(Dropout(0.8))
modelBin.add(Dense(2,activation='softmax'))

#Compile model
adam=Adam(lr=0.0001)
modelBin.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

#Train the model
modelBin.fit(featuresPad,labelsBinVector,batch_size=20,nb_epoch=30,validation_split=0.1)

Train on 16762 samples, validate on 1863 samples
Epoch 1/30
16762/16762 [==============================] - 237s - loss: 0.5631 - acc: 0.7573 - val_loss: 0.4365 - val_acc: 0.8588
Epoch 2/30
16762/16762 [==============================] - 237s - loss: 0.5394 - acc: 0.7587 - val_loss: 0.4268 - val_acc: 0.8588
Epoch 3/30
16762/16762 [==============================] - 234s - loss: 0.4892 - acc: 0.7680 - val_loss: 0.4173 - val_acc: 0.8551
Epoch 4/30
16762/16762 [==============================] - 234s - loss: 0.4368 - acc: 0.8007 - val_loss: 0.4507 - val_acc: 0.8497
Epoch 5/30
16762/16762 [==============================] - 234s - loss: 0.4011 - acc: 0.8235 - val_loss: 0.4840 - val_acc: 0.8293
Epoch 6/30
16762/16762 [==============================] - 234s - loss: 0.3720 - acc: 0.8390 - val_loss: 0.5139 - val_acc: 0.8041
Epoch 7/30
16762/16762 [==============================] - 234s - loss: 0.3530 - acc: 0.8491 - val_loss: 0.5373 - val_acc: 0.8229
Epoch 8/30
16762/16762 [========================